In [ ]:
# to use cd, etc
import sys
import os

import numpy as np
# for math work
from numpy import linalg, matlib
# save, import and load data
from numpy import asarray, save, load

# needed for some matrix manipulations
import cvxpy as cp
# import math 

import scipy
# for curve fitting
from scipy import optimize
from scipy.io import loadmat  # this is the SciPy module that loads mat-files

from sklearn.metrics import r2_score, mean_squared_error

import mat73
import pandas as pd

import matplotlib.pyplot as plt
# for plots
import seaborn
# set up seaborn for the plots
seaborn.set()

# suppress deprecation warnings
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# # # to save data plots
# from matplotlib.backends.backend_pdf import PdfPages

# from datetime import datetime
import pickle

### Set start and end days for session

In [ ]:
sessStart = '072312'
sessEnd = '080412'

### Set up all paths

In [ ]:
nDir = 'catNeuralDat_jeev' + sessStart + '_' + sessEnd + '_trE5_B100.mat'
kinDir = 'catBehaviorDat_jeev' + sessStart + '_' + sessEnd + '.mat' #catBehaviorDat_jeev072312_080412
decDir = 'decoderParams_jeev'+ sessStart + '_' + sessEnd + '.mat'
pathDir = '/Volumes/GoogleDrive/Shared drives/aoLab/Data/bmiLearning_jeev/jeev' + sessStart + '_' + sessEnd
os.chdir(pathDir)
os.getcwd()

In [ ]:
nPath = pathDir + '/' + nDir
nDict = mat73.loadmat(nPath, use_attrdict=True)
nDict.keys()

### Loading kinematic (cursor) data

In [ ]:
# kinPath = pathDir + '/' + kinDir
# kinPath

In [ ]:
# kinPath = pathDir + '/' + kinDir
# kinDict = mat73.loadmat(kinPath, use_attrdict=True)
# kinDict.keys()

### Loading Decoder Data

In [ ]:
decPath = pathDir + '/' + decDir
decPath

In [ ]:
decPath = pathDir + '/' + decDir
decDict = mat73.loadmat(decPath, use_attrdict=True)
decDict.keys()

### Sorting n_BMI to get time bin x decoder neurons x successful trials

#### Select Decoder Number

In [ ]:
decNum = 2

#### Select time bins to care about

In [ ]:
# time bins
alignCue = 20
endCue = len(nDict['binvector']) -1 

In [ ]:
nDict['binvector']

In [ ]:
nDict['bmiDecoderInd']

#### Find Units Used For this Decoder

In [ ]:
# all units
decDict['bmiUnitInds_all']

In [ ]:
usedUnits = (decDict['bmiUnitInds_all'][decDict['usedUnit_decoder'][decNum]] - 1).astype(int)
usedUnits

#### Find Successful Trials for that decoder in this series

In [ ]:
# sessions that we care about
sessInd = np.where(nDict['bmiDecoderInd'] == decNum)[0] + 1
decTrials = np.where((nDict['sessID_bmi'] >= sessInd[0]) & (nDict['sessID_bmi'] <= sessInd[-1]))[0]
decTrials.shape

In [ ]:
sessInd

In [ ]:
succCol = 6
dS = decTrials[0]
dE = decTrials[-1]
succIdx = (np.where((nDict['trEs_bmi'][dS:dE, succCol] == 8) | (nDict['trEs_bmi'][dS:dE, succCol] == 9))[0]) + dS

In [ ]:
succIdx 

In [ ]:
nData_all = nDict['N_norm_bmi'][alignCue:endCue, usedUnits, :]
nData_all.shape

In [ ]:
nData_succ  = nData_all[:, :, succIdx]

In [ ]:
nData_succ.shape

In [ ]:
plt.plot(np.mean(nData_succ, 2));
plt.title("Average Firing Rate Across Decoder Trials (not standardized)")
plt.xlabel("Time Bins (From Go Cue)")
plt.ylabel("Firing Rates (Hz)")

### Saving Data

In [ ]:
# date = datetime.today().strftime('%Y_%m_%d')
savepath = '/Users/mmadduri/Documents/PhD/ResearchMaterials/Code/BMI_Model/data/' + 'jeev' + sessStart + '_' + sessEnd + '/'
# savepath

try:
    os.makedirs(savepath) # create directory [current_path]/feed/address
except OSError:
    pass # already exists

In [ ]:
data = [decNum, usedUnits, succIdx, nData_succ]
with open(savepath + 'decoder_' + str(decNum) + '_neuralData.pkl', 'wb') as outfile:
    pickle.dump(data, outfile, pickle.HIGHEST_PROTOCOL)